In [11]:
from math import sqrt

critics = {
    '김지훈': {
        '우동가조쿠': 4.5,
        '행운돈까스': 3.5,
        '신소재공학관 교직원식당': 2.0,
        '맥도날드':4.0,
        '종로 빈대떡': 3.0,
    },

    '권현우': {
        '우동가조쿠': 3.0,
        '봉구스밥버거': 5.0,
        '금룡': 4.5,
    },

    '박준형': {
        '우동가조쿠': 5.0,
        '행운돈까스': 4.3,
        '와와':4.8,        
        '봉구스밥버거': 5.0,
    },

    '정승환': {
        '행운돈까스': 4.9,
        '신소재공학관 교직원식당': 4.0,
        '와와':4.8,        
        '봉구스밥버거': 4.0,
        '금룡':3.5
        
    },

    '홍순현': {
        '우동가조쿠': 5.0,
        '신소재공학관 교직원식당': 4.0,
        '봉구스밥버거': 4.0,
        '종로 빈대떡': 4.0,
        '금룡' : 4.0,
        '와와':4.0,
        '맥도날드':3.5,
        '행운돈까스':4.0
    },

    '강태욱': {
        '우동가조쿠': 4.8,
        '행운돈까스': 2.0,
        '와와':1.4,        
        '봉구스밥버거': 2.0,
        '금룡': 2.5,
    },

    '함규식': {
        '행운돈까스': 4.5,
        '신소재공학관 교직원식당': 4.0,
        '금룡': 3.5,
        '봉구스밥버거': 4.5,
        '고져스키친':1.0,
    },
 
    '전민종': {
        '우동가조쿠': 3.0,
        '행운돈까스': 1.0,
        '신소재공학관 교직원식당': 1.0,
        '봉구스밥버거': 2.0,
        '금룡': 1.0,
    },
    
    '윤영록': {
        '금룡':3.5,
        '고져스키친': 4.0,
        '행운돈까스':3.0,
    },
    
    '김규민': {
        '우동가조쿠': 3.5,
        '행운돈까스': 5.0,
        '신소재공학관 교직원식당': 2.0,
    },
    
   
}


# Euclidean Distance
def sim_distance(prefs, person1, person2):
    # 공통 항목 추출
    si = dict()

    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1

    # 공통 평가 항목이 없는 경우 0 리턴
    if len(si) == 0: return 0

    # person1의 item이 person2에서도 존재한다면, person1과 person2의 item 평점 차이의 제곱한 값을 더한 후 제곱 근을 계산
    sum_of_squares = sum([(prefs[person1][item] - prefs[person2][item])**2 for item in prefs[person1] if item in prefs[person2]])

    return 1/(1+sqrt(sum_of_squares))

# Pearson correlation coefficient
def sim_pearson(prefs, p1, p2):
    # 같이 평가한 항목들의 목록을 구함
    si = dict()

    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1

    # 공통 항목 개수
    n = len(si)

    # 공통 항목이 없으면 0 리턴
    if n==0: return 0

    # 모든 선호도를 합산
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    # 제곱의 합을 계산
    sum1Sq = sum([(prefs[p1][it])**2 for it in si])
    sum2Sq = sum([(prefs[p2][it])**2 for it in si])

    # 곱의 합을 계산
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])

    # 피어슨 점수 계산
    num = pSum - (sum1*sum2/n)
    den = sqrt((sum1Sq-pow(sum1,2)/n) * (sum2Sq-pow(sum2,2)/n))
    if den==0: return 0

    r = num/den

    return r


In [13]:

# 선호도 dict에 최적의 상대편을 구함
# 결과 개수와 유사도 함수는 옵션
def top_matches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other) for other in prefs if other!=person]

    scores.sort()
    scores.reverse()
    return scores[:n]

# 다른 사람과의 순위의 가중 평균값을 이용해서 특정 사람을 추천
def get_recommendations(prefs, person, similarity=sim_pearson):
    totals = dict()
    simSums = dict()
    
    for other in prefs:
        # 나를 제외 하고
        if other == person: continue
        sim = similarity(prefs, person, other)  # person과 other 사이의 상관계수 점수를 구함

        # 0 이하 점수는 무시
        if sim<=0: continue

        for item in prefs[other]:   # ohter가  가본 식당들의 list
            # 내가 가보지 못한 식당만 대상
            if item not in prefs[person] or prefs[person][item] == 0:
                # 유사도 * 점수
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item]*sim  # other가 평가한 식당의 점수 * person과 other의 상관계수

                # 유사도 합계
                simSums.setdefault(item, 0)
                simSums[item] += sim
             
    #1 사용자가 모든 식당을 이용했을 경우//  사용자가 DB에 있는 모든 식당을 이용하고 평가를 했다면, 추천 해줄 식당이 없다. !
    #2 상관계수 점수가 0보다 작은 사람만 있을 경우도 추천 해줄 식당이 없다.
    #3  1 + 2의 경우가 합쳐진 경우
   
    # 정규화된 목록 생성
    rankings = [ (total/simSums[item], item) for item, total in totals.items() ]
    
    #반환할 목록이 없다면 에러메세지 출력하고 종료
    if rankings ==[]:
        error_message ="죄송합니다. 데이터 부족으로 인해서 , 추천 서비스를 이용하실 수 없습니다.  업데이트를 기다려 주세요.^^"
        return error_message

    # 정렬된 목록 리턴
    rankings.sort()
    rankings.reverse()
    
    
    
    #추정 평가가 3.5점 이상인 식당만 반환
    new_rankings=[]
    for item in rankings:
        if item[0] > 3.5 :
             new_rankings.append(item)
                
    #3.5 이상의 추천 맛집이  없을 경우
    if new_rankings ==[]:
        error_message =" 사용자님이 원하실만한 식당이 부족합니다..  업데이트를 기다려주세요.^^"
        return error_message
    
    return new_rankings

# 사람을 제품 기준으로 dict 변경
def transform_prefs(prefs):
    result = dict()

    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, dict())

            result[item][person] = prefs[person][item]

    return result

In [15]:
#유사거리
for item in critics: 
     print ("정승환 and",  item, sim_distance(critics, '정승환', item))



정승환 and 김지훈 0.2905869199892622
정승환 and 권현우 0.4142135623730951
정승환 and 박준형 0.4616399415807224
정승환 and 정승환 1.0
정승환 and 홍순현 0.4340578300578996
정승환 and 강태욱 0.1667500667275592
정승환 and 함규식 0.6096399597571442
정승환 and 전민종 0.14555481316723456
정승환 and 윤영록 0.3448275862068965
정승환 and 김규민 0.3330559599501922


In [17]:
#피어슨 상관계수

for item in critics:
    print ("정승환", item, sim_pearson(critics, "정승환", item))



정승환 김지훈 0.9999999999999997
정승환 권현우 1.0
정승환 박준형 -0.7871464784293108
정승환 정승환 1.0
정승환 홍순현 0
정승환 강태욱 -0.7758405947503793
정승환 함규식 0.7762058642344947
정승환 전민종 -0.11433239009500695
정승환 윤영록 -0.9999999999999919
정승환 김규민 0.999999999999997


In [19]:
#나와 유사 평론가 순위
for score, other in top_matches(critics, "정승환"): 
    print ("정승환 and", other, score)



정승환 and 권현우 1.0
정승환 and 김지훈 0.9999999999999997
정승환 and 김규민 0.999999999999997
정승환 and 함규식 0.7762058642344947
정승환 and 홍순현 0


In [20]:
#소수점 2번쨰 자리까지
for item in [item for item in critics if item!="정승환"]: 
    print ("정승환 and", item, round(sim_pearson(critics, "정승환", item),2))


정승환 and 김지훈 1.0
정승환 and 권현우 1.0
정승환 and 박준형 -0.79
정승환 and 홍순현 0
정승환 and 강태욱 -0.78
정승환 and 함규식 0.78
정승환 and 전민종 -0.11
정승환 and 윤영록 -1.0
정승환 and 김규민 1.0


In [23]:
 #항목 추천, 피어슨 상관계수 기반
for score, item in get_recommendations(critics, "정승환"): 
    print (item, round(score,2))



맥도날드 3.84


In [25]:
#항목 추천, 유클리디안 거리 기반
for score, item in get_recommendations(critics, "정승환", similarity=sim_distance): 
    print (item, round(score,2))



우동가조쿠 4.2
맥도날드 3.7
종로 빈대떡 3.6


In [490]:
#제품 추천, // 냉면을 좋아하는 사람은 라면이나 짬뽕을 좋아하는 경향이 있다, 
transformed = transform_prefs(critics) 
for score, item in top_matches(transformed, "와와"): 
    print (item, round(score,2))



행운돈까스 0.98
우동가조쿠 0.97
봉구스밥버거 0.95
금룡 0.85
종로 빈대떡 0


In [26]:
#총 출력 결과

print(get_recommendations(critics, "김지훈"))
print(get_recommendations(critics, "박준형"))
print(get_recommendations(critics, "정승환"))
print(get_recommendations(critics, "홍순현"))
print(get_recommendations(critics, "권현우"))
print(get_recommendations(critics, "전민종"))
print(get_recommendations(critics, "함규식"))
print(get_recommendations(critics, "강태욱"))
print(get_recommendations(critics, "윤영록"))
print(get_recommendations(critics, "김규민"))


[(3.704112527987195, '와와'), (3.5936590293433426, '봉구스밥버거')]
[(3.843881332669246, '맥도날드')]
[(4.0, '맥도날드'), (3.666666666666667, '우동가조쿠')]
죄송합니다. 데이터 부족으로 인해서 , 추천 서비스를 이용하실 수 없습니다.  업데이트를 기다려 주세요.^^
[(4.8, '와와'), (4.7, '행운돈까스'), (4.0, '신소재공학관 교직원식당')]
[(3.7392659623604487, '맥도날드'), (3.521468075279102, '종로 빈대떡')]
[(4.8, '와와'), (4.0, '맥도날드'), (3.567821611530458, '우동가조쿠')]
[(4.0, '고져스키친'), (3.755679863711353, '맥도날드')]
[(4.8, '우동가조쿠')]
[(4.8, '와와'), (4.25, '봉구스밥버거'), (4.0, '맥도날드')]


In [35]:
# 취향 저격 식당 보여주기
def show_recommendations(person, prefs = critics, n=1) :
    item = get_recommendations(prefs, person)
    if not type(item) == str:
          for x in range(len(item)):
            print(n, item[x][1])
            n+=1
    else: print(item)
    

In [43]:
show_recommendations('정승환')

1 맥도날드
2 우동가조쿠
